In [1]:
# importing modules
import pandas as pd
import numpy as np
import pandas as pd
import geopandas as gpd
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show
from bokeh.models import GeoJSONDataSource, ColumnDataSource, HoverTool
from bokeh.palettes import Viridis256, Inferno256, Magma256
from bokeh.palettes import BrBG, PiYG, RdGy, RdYlGn, YlGnBu
from bokeh.models import LinearColorMapper, ColorBar
#import streamlit as st
# #from streamlit_folium import st_folium
#import folium
#from leaflet import Map, GeoJSON

In [2]:
# Reading the data
# rheas_unimodal=pd.read_csv('data/tza_n_25_tamsat_unimodal_2015_2023_maize.csv')
# rheas_bimodal=pd.read_csv('data/tza_n_25_bimodal_2015_2023_maize.csv')
rheas_unimodal=pd.read_csv('data/tza_n_25_tamsat_unimodal_newmask_2022_2023_maize.csv')
rheas_bimodal=pd.read_csv('data/tza_n_25_tamsat_bimodal_newmask_2022_2023_maize.csv')



In [3]:
# change dates to months or years
# unimodal
rheas_unimodal['planting']=pd.to_datetime(rheas_unimodal['planting'], format='ISO8601')
rheas_unimodal['harvest']=pd.to_datetime(rheas_unimodal['harvest'], format='ISO8601')
rheas_unimodal['harvest']=pd.to_datetime(rheas_unimodal['harvest'], format='ISO8601')
rheas_unimodal['year'] = rheas_unimodal['planting'].dt.year
rheas_unimodal['year'] = rheas_unimodal['planting'].dt.year
rheas_unimodal['yield']=rheas_unimodal['gwad']/1000
# rheas_raw['harvest']=pd.to_datetime(rheas_raw['harvest'], format='%Y/%m/%d')

# bimodal
rheas_bimodal['planting']=pd.to_datetime(rheas_bimodal['planting'], format='ISO8601')
rheas_bimodal['harvest']=pd.to_datetime(rheas_bimodal['harvest'], format='ISO8601')
rheas_bimodal['harvest']=pd.to_datetime(rheas_bimodal['harvest'], format='ISO8601')
rheas_bimodal['year'] = rheas_bimodal['planting'].dt.year
rheas_bimodal['year'] = rheas_bimodal['planting'].dt.year
rheas_bimodal['yield']=rheas_bimodal['gwad']/1000

In [4]:
# mask off water bodies eg Lake Victoria
districts1=['Kati','Wete', 'Magharibi','Micheweni','Kahama Township Authority','Kahama Township Authority',
            'Korogwe Township Authority',"Kaskazini 'A'","Kaskazini 'B'",'Handeni Township Authority']


def exclude_districts(dataframe,districts):
    set2=set(list(dataframe.cname.unique()))-set(districts)
    districts2=list(set2)
    list1=[]
    for d in districts2:
        df1=dataframe.loc[dataframe['cname']==d]
        list1.append(df1)
    df2=pd.concat(list1) 
    return df2

rheas_unimodal=exclude_districts(dataframe=rheas_unimodal,districts=districts1)
rheas_bimodal=exclude_districts(dataframe=rheas_bimodal,districts=districts1)
rheas_bimodal.cname.unique()

array(['Babati Urban', 'Bukombe', 'Gairo', 'Muleba', 'Mvomero', 'Arusha',
       'Mbogwe', 'Busega', 'Serengeti', 'Ngara', 'Kwimba', 'Kilindi',
       'Mwanga', 'Maswa', 'Kibaha', 'Kilosa', 'Hai', 'Handeni', 'Bunda',
       'Tarime', 'Kyerwa', 'Kibaha Urban', 'Bagamoyo', 'Arusha Urban',
       'Bukoba Rural', 'Mbulu', 'Same', 'Kishapu', 'Bariadi', 'Sengerema',
       'Kisarawe', 'Musoma Rural', 'Kiteto', 'Ilemela', 'Hanang', 'Chato',
       'Morogoro Rural', 'Korogwe', 'Nyamagana', 'Kahama', 'Magu',
       'Lushoto', 'Kakonko', 'Simanjiro', 'Shinyanga Urban', 'Siha',
       'Missenyi', 'Shinyanga Rural', 'Kinondoni', 'Longido', 'Monduli',
       'Morogoro Urban', 'Pangani', 'Itilima', 'Meatu', 'Moshi Urban',
       'Misungwi', 'Rombo', 'Geita', 'Biharamulo', 'Moshi Rural', 'Meru',
       'Ngorongoro', 'Musoma Urban', 'Temeke', 'Karagwe', 'Butiama',
       "Nyang'wale", 'Mkuranga', 'Babati', 'Rorya', 'Ilala', 'Muheza',
       'Karatu', 'Ukerewe', 'Mkinga', 'Tanga'], dtype=object)

In [5]:
#unimodal (Long Rains) season [9,10,11,12, 1,2,3,4]
#  unimodal_season1=[1,2, 3, 4], unimodal_season1 = [9, 10, 11, 12]
# unimodal part1
rheas_unimodal1=rheas_unimodal
unimodal_season1 = ['unimodal', 'unimodal', 'unimodal', 'unimodal','none', 'none', 'none', 
                   'none', 'none', 'none', 'none', 'none']
unimodal_month_to_season1=dict(zip(range(1,13), unimodal_season1))
rheas_unimodal1['season']= rheas_unimodal1['planting'].dt.month.map(unimodal_month_to_season1)

# unimodal part2
rheas_unimodal2=rheas_unimodal
unimodal_season2 = ['none', 'none', 'none', 'none','none', 'none', 'none', 
                   'none', 'unimodal', 'unimodal', 'unimodal', 'unimodal']
unimodal_month_to_season2=dict(zip(range(1,13), unimodal_season2))
rheas_unimodal2['season']= rheas_unimodal2['planting'].dt.month.map(unimodal_month_to_season2)


In [6]:
# bimodal MAM and OND seasons MAM =[1, 2, 3, 4], OND =[9, 10, 11]
bimodal_season = [1, 2, 3, 4]
bimodal_season= ['mam', 'mam', 'mam', 'mam','none', 'none',
                       'none', 'none', 'ond', 'ond', 'ond', 'none']
bimodal_month_to_season=dict(zip(range(1,13), bimodal_season))
rheas_bimodal['season']= rheas_bimodal['planting'].dt.month.map(bimodal_month_to_season)


In [7]:
# # Extract data only for the seasons
# long rains (unimodal)
rheas_unimodal1= rheas_unimodal1.loc[rheas_unimodal1['season']=='unimodal']
rheas_unimodal2= rheas_unimodal2.loc[rheas_unimodal2['season']=='unimodal']

# bimodal MAM and OND
rheas_bimodal_mam=rheas_bimodal.loc[rheas_bimodal['season']=='mam']
rheas_bimodal_ond=rheas_bimodal.loc[rheas_bimodal['season']=='ond']


In [8]:

# extract data for 2022 and 2023 only
# unimodal
rheas_unimodal_2022_2=rheas_unimodal2.loc[rheas_unimodal2['year']==2022].reset_index()
rheas_unimodal_2023_1=rheas_unimodal1.loc[rheas_unimodal1['year']==2023].reset_index()
rheas_unimodal_2022=pd.concat([rheas_unimodal_2022_2,rheas_unimodal_2023_1],axis=0)

# bimodal
rheas_bimodal_mam_2022=rheas_bimodal_mam.loc[rheas_bimodal_mam['year']==2022]
rheas_bimodal_mam_2023=rheas_bimodal_mam.loc[rheas_bimodal_mam['year']==2023]
rheas_bimodal_ond_2022=rheas_bimodal_ond.loc[rheas_bimodal_ond['year']==2022]

In [9]:
# # aggregate 
rheas_unimodal_2022=rheas_unimodal_2022.groupby(['cname'], as_index=False).agg({'yield': 'mean'})
rheas_bimodal_mam_2022=rheas_bimodal_mam_2022.groupby(['cname'], as_index=False).agg({'yield': 'mean'})
rheas_bimodal_mam_2023=rheas_bimodal_mam_2023.groupby(['cname'], as_index=False).agg({'yield': 'mean'})
rheas_bimodal_ond_2022=rheas_bimodal_ond_2022.groupby(['cname'], as_index=False).agg({'yield': 'mean'})
rheas_bimodal_ond_2022['yield'].min()

2.971375

In [10]:
# Read shapefiles and rename columns
country_shapefile = gpd.read_file("shp/gadm36_TZA_0.shp")
# tanzania_unimodal_shp = gpd.read_file("../../../../gadm36_TZA_2_Unimodal.shp")
# tanzania_bimodal_shp = gpd.read_file("../../../../gadm36_TZA_2_Bimodal.shp")

tanzania_unimodal_shp = gpd.read_file("shp/tza_unimodal.shp")
tanzania_bimodal_shp = gpd.read_file("shp/tza_bimodal.shp")


# Rename kenya_shp 
tanzania_unimodal_shp=tanzania_unimodal_shp.rename(columns={'NAME1':'cname'})
tanzania_bimodal_shp=tanzania_bimodal_shp.rename(columns={'NAME1':'cname'})
# # tanzania_bimodal_shp=tanzania_unimodal_shp.rename(columns={'NAME_2':'cname'})
# tanzania_unimodal_shp.to_file('../../../../shp/tza_unimodal.shp')
# tanzania_bimodal_shp.to_file('../../../../shp/tza_bimodal.shp')
# tanzania_bimodal_shp['cname']

# tanzania_unimodal_shp.columns
# # tanzania_unimodal_shp.cname.unique()
# tanzania_unimodal_shp.cname.unique()
# rheas_unimodal.cname.unique()
# # tanzania_bimodal_shp.cname.unique()

In [11]:
# Merge data into shapefiles
unimodal_2022 = pd.merge(tanzania_unimodal_shp, rheas_unimodal_2022, on="cname")
bimodal_mam_2022= pd.merge(tanzania_bimodal_shp, rheas_bimodal_mam_2022, on="cname")
bimodal_mam_2023= pd.merge(tanzania_bimodal_shp, rheas_bimodal_mam_2023, on="cname")
bimodal_ond_2022= pd.merge(tanzania_bimodal_shp, rheas_bimodal_ond_2022, on="cname")
unimodal_2022['yield'].min()

2.952

In [12]:
# create list of min and max values
list1=[unimodal_2022['yield'].min(),unimodal_2022['yield'].max(),
       bimodal_mam_2022['yield'].min(),bimodal_mam_2022['yield'].max(),
       bimodal_mam_2023['yield'].min(),bimodal_mam_2023['yield'].max(),bimodal_ond_2022['yield'].min(),bimodal_ond_2022['yield'].max()]
value=(list1)

In [13]:
geo_source_unimodal_2022 = GeoJSONDataSource(geojson=unimodal_2022.to_json())
geo_source_mam_2022= GeoJSONDataSource(geojson=bimodal_mam_2022.to_json())
geo_source_mam_2023= GeoJSONDataSource(geojson=bimodal_mam_2023.to_json())
geo_source_ond_2022= GeoJSONDataSource(geojson=bimodal_ond_2022.to_json())
country_geo_source = GeoJSONDataSource(geojson=country_shapefile.to_json())

In [14]:
color_mapper = LinearColorMapper(palette="RdBu10", low=min(value).round(1), high=max(value).round(1))
p1 = figure(title='Unimodal_Long Rains Season Yield (mt/ha) - 2022',
           outer_width=350, outer_height=300)

p1.multi_line('xs','ys', source=country_geo_source,
            line_color='black', line_width=0.5)

p1.patches('xs','ys', source=geo_source_unimodal_2022,
          fill_color={'field': 'yield', 'transform': color_mapper},
          fill_alpha=0.7, line_color='black', line_width=0.5)

hover = HoverTool(tooltips=[('District: ', '@cname'), ('Yield', '@yield')])
p1.add_tools(hover)

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=20, height=300,
                     border_line_color=None, location=(0,0), orientation='vertical')
p1.add_layout(color_bar, 'right')

In [15]:
color_mapper = LinearColorMapper(palette="RdBu10", low=min(value).round(1), high=max(value).round(1))
p2 = figure(title='Bimodal_MAM Season Yield (mt/ha) - 2022',
           outer_width=350, outer_height=300)

p2.multi_line('xs','ys', source=country_geo_source,
            line_color='black', line_width=0.5)

p2.patches('xs','ys', source=geo_source_mam_2022,
          fill_color={'field': 'yield', 'transform': color_mapper},
          fill_alpha=0.7, line_color='black', line_width=0.5)

hover = HoverTool(tooltips=[('District: ', '@cname'), ('Yield', '@yield')])
p2.add_tools(hover)

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=20, height=300,
                     border_line_color=None, location=(0,0), orientation='vertical')
p2.add_layout(color_bar, 'right')

In [16]:
# Bimodal Season 2022
color_mapper1 = LinearColorMapper(palette="RdBu10", low=min(value).round(1), high=max(value).round(1))
p3 = figure(title=' Bimodal_ MAM Season  Yield(mt/Ha) - 2023',
           outer_width=350, outer_height=300)

p3.multi_line('xs','ys', source=country_geo_source,
            line_color='black', line_width=0.5)

p3.patches('xs','ys', source=geo_source_mam_2023,
          fill_color={'field': 'yield', 'transform': color_mapper1},
          fill_alpha=0.7, line_color='black', line_width=0.5)

hover = HoverTool(tooltips=[('District', '@cname'), ('Yield', '@yield')])
p3.add_tools(hover)

color_bar = ColorBar(color_mapper=color_mapper1, label_standoff=8, width=20, height=300,
                     border_line_color=None, location=(0,0), orientation='vertical')
p3.add_layout(color_bar, 'right')

In [17]:
# Bimodal Season 2022
color_mapper1 = LinearColorMapper(palette="RdBu10", low=min(value).round(1), high=max(value).round(1))
p4 = figure(title='Bimodal_OND Season  Yield(mt/ha) - 2022',
           outer_width=350, outer_height=300)

p4.multi_line('xs','ys', source=country_geo_source,
            line_color='black', line_width=0.5)

p4.patches('xs','ys', source=geo_source_ond_2022,
          fill_color={'field': 'yield', 'transform': color_mapper1},
          fill_alpha=0.7, line_color='black', line_width=0.5)

hover = HoverTool(tooltips=[('District', '@cname'), ('Yield', '@yield')])
p4.add_tools(hover)

color_bar = ColorBar(color_mapper=color_mapper1, label_standoff=8, width=20, height=300,
                     border_line_color=None, location=(0,0), orientation='vertical')
p4.add_layout(color_bar, 'right')

In [18]:

grid=gridplot([ p2, p3, p4, p1],ncols=3, width=650, height=600)
# grid = gridplot([[p2, p3], [None, p4]], width=650, height=600)
show(grid)

In [19]:
from bokeh.io import save
# # # specify the output file name and location
save(grid, filename='../tanzania.html', title='Maize Yield Maps')

C:\Users\kasit\AppData\Local\Temp\ipykernel_16528\1104631151.py:3: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  save(grid, filename='../tanzania.html', title='Maize Yield Maps')


'd:\\RONO\\rheas\\maize_crop_modeling\\crop-modeling\\tanzania.html'